In [1]:
import numpy as np

In [25]:
def tactile(x, y, M=40, N=20, d=0.5, I_0=2.5, sigma_I=0.3):
    region = np.zeros((M,N))
    
    for i in range(len(region)):
        for j in range(len(region[i])):
            I = I_0 * np.exp( - ((x - 0.5*i)**2 + (y - 0.5*j)**2) / (2 * (sigma_I**2)))
            
            region[i][j] = I
    
    return region
    
def auditory(M=20, N=3, d=10):
    region = np.zeros((M,N))

In [27]:
tactile(0.2, 0.2)

array([[1.60295097e+000, 1.21417946e+000, 5.71836623e-002,
        1.67450901e-004, 3.04880361e-008, 3.45142031e-013,
        2.42936455e-019, 1.06319755e-026, 2.89308828e-035,
        4.89481006e-045, 5.14916063e-056, 3.36793325e-068,
        1.36967318e-081, 3.46335384e-096, 5.44506686e-112,
        5.32274927e-129, 3.23515610e-147, 1.22259020e-166,
        2.87271816e-187, 4.19692849e-209],
       [1.21417946e+000, 9.19698603e-001, 4.33146301e-002,
        1.26838218e-004, 2.30936242e-008, 2.61433053e-013,
        1.84015893e-019, 8.05335071e-027, 2.19141349e-035,
        3.70764793e-045, 3.90030962e-056, 2.55109199e-068,
        1.03747967e-081, 2.62336976e-096, 4.12444827e-112,
        4.03179696e-129, 2.45051793e-147, 9.26069444e-167,
        2.17598383e-187, 3.17902698e-209],
       [5.71836623e-002, 4.33146301e-002, 2.03996959e-003,
        5.97364232e-006, 1.08763000e-009, 1.23125945e-014,
        8.66651348e-021, 3.79285025e-028, 1.03208012e-036,
        1.74617421e-046, 1.83